In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score

from src.data import etl

In [11]:
dir_df_mdl = '../../data/processed/model/df_mdl.csv'
df = pd.read_csv(dir_df_mdl, sep=';')
df.head()

,datgeracaoconjuntodados,nomagente,dthinicioocorrenciaaberta,dthfimocorrenciaaberta,ufibge,nom_ufibge,ocorrencia_meio_ambiente,data_key,uf,pressao_atm,temp_min,temp_max,umidade,vento_direcao,vento_rajada,vento_velocidade
0,2024-09-10,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2024-01-01 03:27:44,2024-01-01 04:18:40,12,Acre,False,2024-01-01 03:00:00,AC,986.8,24.2,24.7,48.5,33.5,2.5,0.2
1,2024-09-10,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2024-01-01 06:09:00,2024-01-01 09:06:08,12,Acre,True,2024-01-01 06:00:00,AC,985.6,23.6,23.7,49.0,19.0,1.0,0.3
2,2024-09-10,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2024-01-01 06:21:00,2024-01-01 11:43:29,12,Acre,False,2024-01-01 06:00:00,AC,985.6,23.6,23.7,49.0,19.0,1.0,0.3
3,2024-09-10,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2024-01-01 06:25:00,2024-01-01 07:39:46,12,Acre,False,2024-01-01 06:00:00,AC,985.6,23.6,23.7,49.0,19.0,1.0,0.3
4,2024-09-10,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2024-01-01 06:30:00,2024-01-01 10:48:56,12,Acre,True,2024-01-01 06:00:00,AC,985.6,23.6,23.7,49.0,19.0,1.0,0.3


In [ ]:
variaveis = [
    'ufibge', 
    'pressao_atm', 
    'temp_min', 
    'temp_max', 
    'umidade', 
    'vento_direcao', 
    'vento_rajada', 
    'vento_velocidade'
]

x = df[variaveis]
y = df['ocorrencia_meio_ambiente']

In [ ]:
# Dividindo os dados em conjunto de treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(
    x, 
    y, 
    test_size=0.2, 
    random_state=100
)

In [ ]:
# Criando o modelo LightGBM com a API do scikit-learn
model = lgb.LGBMClassifier(
    objective='binary', 
    learning_rate=0.025,
    num_leaves=32,
    max_depth=8,
    n_estimators=1000,
    early_stopping_rounds=10,
    verbose=-1
)

In [ ]:
# Treinando o modelo
model.fit(
    x_treino, 
    y_treino, 
    eval_set=[(x_teste, y_teste)]
)

In [2]:
# Estimado
predito = model.predict(x_teste)
predito_prob = [p[1] for p in model.predict_proba(x_teste)]

# Métricas
accuracy = accuracy_score(y_teste, predito)
auc = roc_auc_score(y_teste, predito_prob)
precision = precision_score(y_teste, predito)

# Exibindo as métricas
print(f'Acurácia do modelo: {accuracy:.4f}')
print(f'AUC do modelo: {auc:.4f}')
print(f'Precisão do modelo: {precision:.4f}')

Acurácia do modelo: 0.85
AUC do modelo: 0.78
Precisão do modelo: 0.83


In [16]:
df['probabilidade'] = [p[1] for p in model.predict_proba(df[variaveis])]

In [32]:
table_id = 'alert-being-435923-n9.modelo.nimbus'
etl.exporta_dados_bq(df, table_id)